# Descripción del proyecto
Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.  

Necesitamos predecir si un cliente dejará el banco pronto. Tú tienes los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.  

Crea un modelo con el máximo valor *F1* posible. Para aprobar la revisión, necesitas un valor *F1* de al menos 0.59. Verifica F1 para el conjunto de prueba.  

Además, debes medir la métrica *AUC-ROC* y compararla con el valor *F1*.

## Instrucciones del proyecto
1. Descarga y prepara los datos.  Explica el procedimiento.
2. Examina el equilibrio de clases. Entrena el modelo sin tener en cuenta el desequilibrio. Describe brevemente tus hallazgos.
3. Mejora la calidad del modelo. Asegúrate de utilizar al menos dos enfoques para corregir el desequilibrio de clases. Utiliza conjuntos de entrenamiento y validación para encontrar el mejor modelo y el mejor conjunto de parámetros. Entrena diferentes modelos en los conjuntos de entrenamiento y validación. Encuentra el mejor. Describe brevemente tus hallazgos.
4. Realiza la prueba final.

## Descripción de los datos
Puedes encontrar los datos en el archivo  /datasets/Churn.csv file. Descarga el conjunto de datos aqui: https://practicum-content.s3.us-west-1.amazonaws.com/datasets/Churn.csv

### Características

- *RowNumber*: índice de cadena de datos
- *CustomerId*: identificador de cliente único
Surname: apellido
- *CreditScore*: valor de crédito
- *Geography*: país de residencia
- *Gender*: sexo
- *Age*: edad
- *Tenure*: período durante el cual ha madurado el depósito a plazo fijo de un cliente (años)
- *Balance*: saldo de la cuenta
- *NumOfProducts*: número de productos bancarios utilizados por el cliente
- *HasCrCard*: el cliente tiene una tarjeta de crédito (1 - sí; 0 - no)
- *IsActiveMember*: actividad del cliente (1 - sí; 0 - no)
- *EstimatedSalary*: salario estimado

### Objetivo

- *Exited*: El cliente se ha ido (1 - sí; 0 - no)


## Evaluación del proyecto
Hemos definido los criterios de evaluación para el proyecto. Lee esto con atención antes de pasar al ejercicio.

Esto es lo que los revisores buscarán cuando evalúen tu proyecto:

- ¿Cómo preparaste los datos para el entrenamiento? ¿Procesaste todos los tipos de características?
- ¿Explicaste los pasos de preprocesamiento lo suficientemente bien?
- ¿Cómo investigaste el equilibrio de clases?
- ¿Estudiaste el modelo sin tener en cuenta el desequilibrio de clases?
- ¿Qué descubriste sobre la investigación del ejercicio?
- ¿Dividiste correctamente los datos en conjuntos?
- ¿Cómo trabajaste con el desequilibrio de clases?
- ¿Utilizaste al menos dos técnicas para corregir el desequilibrio?
- ¿Realizaste correctamente el entrenamiento, la validación y las pruebas finales del modelo?
- ¿Qué tan alto es tu valor F1?
- ¿Examinaste los valores AUC-ROC?
- ¿Mantuviste la estructura del proyecto y el código limpio?

# Análisis Exploratorio

In [87]:
# Importo mis librerias
import pandas as pd

In [88]:
# Importo y veo mi dataframe

df = pd.read_csv('databases/Churn.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [89]:
# Hago drop a mi columna RowNumber ya que usare el indice que me proporciona pandas
df = df.drop('RowNumber', axis=1)

In [90]:
# Usemos info para ver si los datos son del tipo correcto
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   Surname          10000 non-null  object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              10000 non-null  int64  
 6   Tenure           9091 non-null   float64
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
dtypes: float64(3), int64(7), object(3)
memory usage: 1015.8+ KB


In [91]:
# Busco valores duplicados
print(f'El dataframe tiene una cantidad de {df.duplicated().sum()} valores duplicados')

El dataframe tiene una cantidad de 0 valores duplicados


In [92]:
print(f"Basado en mi funcion .info, la columna 'Tenure' tiene una cantidad de: {df['Tenure'].isna().sum()} valores ausentes.")

Basado en mi funcion .info, la columna 'Tenure' tiene una cantidad de: 909 valores ausentes.


Tengo 909 valores ausentes en 'Tenure', estamos hablando de ~9% de valores ausentes, por lo que procedere a ver como imputarlos.

In [93]:
# Busco el tipo de valores dentro de Tenure
print(df['Tenure'].value_counts().sort_index())

Tenure
0.0     382
1.0     952
2.0     950
3.0     928
4.0     885
5.0     927
6.0     881
7.0     925
8.0     933
9.0     882
10.0    446
Name: count, dtype: int64


Los valores se encuentran entre 0 y 10, imputare mis valores con la media y luego pasare todo a INT. Manteniendo el estilo de que los valores, es decir sin contar fracciones de año, solamente unidades enteras. Al no tener outliers o valores extremos, es mejor imputar con la media en lugar de la mediana.

In [94]:
# Calulo la media y redondeo
tenure_mean= round(df['Tenure'].mean())

# Imputo con la media
df['Tenure'] = df['Tenure'].fillna(tenure_mean)

Ahora que tenemos los datos totalmente imputados, estamos listos para entrenar nuestro primer modelo.

array(['France', 'Spain', 'Germany'], dtype=object)